In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import matplotlib.pyplot as plt
from keras.datasets import mnist
from PIL import Image

### Define some related functions

In [2]:
# activation functions
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1 - np.tanh(x)**2

def ReLU(x):
    return np.maximum(x, 0)

def ReLU_prime(x):
    return np.heaviside(x, 1)

def linear(x):
    return x

def linear_prime(x):
    return 1


# error functions
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def accuracy(y_true, y_pred):
    correct = 0.0
    for i in range(len(y_true)):
        if np.argmax(y_true[i]) == np.argmax(y_pred[i]):
            correct += 1
    return correct / len(y_true)

# loss functions (single example)
def se(y_true, y_pred):
    return (y_true - y_pred)**2

def se_prime(y_true, y_pred):
    return (-2) * (y_true - y_pred)

def softmax(logits):
    exp_logits = np.exp(logits)
    return exp_logits / np.sum(exp_logits)

def softmax_cross_entropy(y_true, y_pred):
    epsilon = 1e-12
    y_pred_softmax = softmax(y_pred)
    #y_pred_softmax = np.clip(y_pred_softmax, epsilon, 1. - epsilon)
    return -1 * np.sum(np.multiply(y_true, np.log(y_pred_softmax)))

def softmax_cross_entropy_prime(y_true, y_pred):
    y_pred_softmax = softmax(y_pred)
    return  y_pred_softmax - y_true

### Define several layers

In [3]:
# Define layers
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input_data):
        raise NotImplementedError("Forward prop not implemented")

    def backprop(self, output_grad, learning_rate):
        raise NotImplementedError("Backward prop not implemented")

In [4]:
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        super().__init__()
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backprop(self, output_grad, learning_rate):
        return np.multiply(self.activation_prime(self.input), output_grad)

In [5]:
class FullyConnectedLayer(Layer):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.weights = np.random.randn(input_size, output_size) / np.sqrt(input_size)  # Xavier initialization
        self.biases = np.zeros((1, output_size))

    def forward(self, input_data):
        self.input = input_data
        self.output = np.dot(input_data, self.weights) + self.biases
        return self.output

    def backprop(self, output_grad, learning_rate):
        input_grad = np.dot(output_grad, self.weights.T)
        weight_grad = np.dot(self.input.T, output_grad)
        bias_grad = np.sum(output_grad, axis=0, keepdims=True)

        self.weights -= learning_rate * weight_grad
        self.biases -= learning_rate * bias_grad

        return input_grad

In [6]:
class FlattenLayer(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, input_data):
        self.input = input_data
        flattened_size = np.prod(input_data.shape)
        self.output = input_data.reshape(1, flattened_size)
        return self.output

    def backprop(self, output_grad, learning_rates):
        input_grad = output_grad.reshape(self.input.shape)
        return input_grad

In [7]:
class ConvolutionalLayer(Layer):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.num_filters = filters
        self.kernel_size = kernel_size
        self.weights = None
        self.bias = None


    def initialize(self, input_shape):
        num_channels = input_shape[0]
        self.weights = np.random.randn(self.num_filters, num_channels, self.kernel_size, self.kernel_size)
        self.bias = np.zeros((self.num_filters, 1))


    def forward(self, input_data):
        self.input = input_data
        self.initialize(self.input.shape)
        num_channels, input_height, input_width = input_data.shape
        output_height = input_height - self.kernel_size + 1
        output_width = input_width - self.kernel_size + 1
        self.output = np.zeros((self.num_filters, output_height, output_width))

        for f in range(self.num_filters):
            for i in range(output_height):
                for j in range(output_width):
                    receptive_field = input_data[:, i: i + self.kernel_size, j:j + self.kernel_size]
                    self.output[f, i, j] = np.sum(receptive_field * self.weights[f]) + self.bias[f]

        return self.output


    def backprop(self, output_grad, learning_rate):
        num_channels, input_height, input_width = self.input.shape
        num_filters, output_height, output_width = self.output.shape

        weight_grad = np.zeros_like(self.weights)
        bias_grad = np.zeros_like(self.bias)
        input_grad = np.zeros_like(self.input)

        for f in range(num_filters):
            for i in range(output_height):
                for j in range(output_width):
                    receptive_field = self.input[:, i: i + self.kernel_size, j: j + self.kernel_size]
                    weight_grad[f] += receptive_field * output_grad[f, i, j]
                    bias_grad[f] += output_grad[f, i, j]
                    input_grad[:, i: i + self.kernel_size, j: j + self.kernel_size] += self.weights[f] * output_grad[f, i, j]

        self.weights -= learning_rate * weight_grad
        self.bias -= learning_rate * bias_grad

        return input_grad

In [8]:
class MaxPoolingLayer(Layer):
    def __init__(self, pool_size):
       super().__init__()
       self.pool_size = pool_size


    def forward(self, input_data):
        self.input = input_data
        num_channels, input_height, input_width = input_data.shape
        pool_height = input_height // self.pool_size
        pool_width = input_width // self.pool_size
        self.output = np.zeros((num_channels, pool_height, pool_width))

        for c in range(num_channels):
            for i in range(pool_height):
                for j in range(pool_width):
                    receptive_field = input_data[c, i*self.pool_size : (i+1)*self.pool_size, j*self.pool_size : (j+1)*self.pool_size]
                    self.output[c, i, j] = np.max(receptive_field)

        return self.output


    def backprop(self, output_grad, learning_rate):
        num_channels, input_height, input_width = self.input.shape
        pool_height = input_height // self.pool_size
        pool_width = input_width // self.pool_size
        input_grad = np.zeros_like(self.input)

        for c in range(num_channels):
            for i in range(pool_height):
                for j in range(pool_width):
                    receptive_field = self.input[c, i*self.pool_size : (i+1)*self.pool_size, j*self.pool_size : (j+1)*self.pool_size]
                    max_value = np.max(receptive_field)
                    mask = (receptive_field == max_value)
                    input_grad[c, i*self.pool_size : (i+1)*self.pool_size, j*self.pool_size : (j+1)*self.pool_size] = mask * output_grad[c, i, j]

        return input_grad

### Define neural network

In [9]:
class Network:
    def __init__(self, loss, loss_prime):
        self.layers = []
        self.loss_func = loss
        self.loss_prime = loss_prime

    def add_layer(self, layer):
        self.layers.append(layer)

    def predict(self, input_data):
        output = input_data
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def train(self, x_train, y_train, epoch_num, lr):
        sample_size = len(x_train)
        error_list = []
        print("==== Start training ====")

        for epoch in range(epoch_num):
            err = 0
            output_list = []
            correct = 0

            # Shuffle the training data and labels
            shuffled_indices = np.random.permutation(sample_size)
            x_train_shuffled = x_train[shuffled_indices]
            y_train_shuffled = y_train[shuffled_indices]

            for i in range(sample_size):
                output = x_train_shuffled[i]

                # Reshape input if it is 1D
                if output.ndim == 1:
                    output = output.reshape(1, -1)

                # Forward propagation
                for layer in self.layers:
                    output = layer.forward(output)

                output_list.append(output)
                err += self.loss_func(y_train_shuffled[i], output)
                print(f"{epoch+1}-{i}th sample")
                print(f"true: {np.argmax(y_train_shuffled[i])}, pred: {np.argmax(output)}")

                # Backpropagation
                grad = self.loss_prime(y_train_shuffled[i], output)
                #print(grad)
                for layer in reversed(self.layers):
                    grad = layer.backprop(grad, lr)

            err /= sample_size
            error_list.append(err)

            print('epoch %d/%d   loss=%f' % (epoch + 1, epoch_num, err))

        return error_list


In [10]:
class Classifier(Network):
    def __init__(self, loss, loss_prime):
        super().__init__(loss, loss_prime)

    def predict(self, input_data):
        sample_size = len(input_data)
        result = []

        for i in range(sample_size):
            output = input_data[i]

            for layer in self.layers:
                output = layer.forward(output)

            output = softmax(output)  # Apply softmax activation
            result.append(output)

        return np.array(result)

In [11]:
# Function for one-hot encoding
def one_hot_encoding(labels, num_classes):
    length = len(labels)
    encoded_labels = np.zeros((length, num_classes))

    for i in range(length):
        encoded_labels[i, int(labels[i])] = 1

    return encoded_labels

### Data preprocessing

In [12]:
# Load the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [13]:
# Filter the training dataset to include only 500 samples for each digit
# Filter the training dataset to include only digits 0 to 5
selected_digits = [0, 1, 2, 3, 4]
filtered_x_train = []
filtered_y_train = []

num_samples_per_digit = 2000

for digit in selected_digits:
    digit_indices = np.where(y_train == digit)[0][:num_samples_per_digit]
    filtered_x_train.append(x_train[digit_indices])
    filtered_y_train.append(y_train[digit_indices])

filtered_x_train = np.concatenate(filtered_x_train, axis=0)
filtered_y_train = np.concatenate(filtered_y_train, axis=0)

In [14]:
# Filter the testing dataset to include only 100 samples for each digit
num_samples_per_digit = 500
filtered_x_test = []
filtered_y_test = []

for digit in selected_digits:
    digit_indices = np.where(y_test == digit)[0][:num_samples_per_digit]
    filtered_x_test.append(x_test[digit_indices])
    filtered_y_test.append(y_test[digit_indices])

filtered_x_test = np.concatenate(filtered_x_test, axis=0)
filtered_y_test = np.concatenate(filtered_y_test, axis=0)

In [15]:
# Expand one dimnesion for channel
x_train, x_test = np.expand_dims(filtered_x_train, 1), np.expand_dims(filtered_x_test, 1)
y_train, y_test = one_hot_encoding(filtered_y_train, 5), one_hot_encoding(filtered_y_test, 5)

# Resize the data
x_train_trans, x_test_trans = np.zeros((len(x_train), 1, 14, 14)), np.zeros((len(x_test), 1, 14, 14))
for idx in range(len(x_train)):
    for i in range(0, 28, 2):
        for j in range(0, 28, 2):
            avg = (int(x_train[idx, 0, i, j])+int(x_train[idx, 0, i+1, j])+int(x_train[idx, 0, i, j+1])+int(x_train[idx, 0, i+1, j+1])) // (2*2)
            x_train_trans[idx, 0, i//2, j//2] = avg

for idx in range(len(x_test)):
    for i in range(0, 28, 2):
        for j in range(0, 28, 2):
            avg = (int(x_test[idx, 0, i, j])+int(x_test[idx, 0, i+1, j])+int(x_test[idx, 0, i, j+1])+int(x_test[idx, 0, i+1, j+1])) // (2*2)
            x_test_trans[idx, 0, i//2, j//2] = avg

In [16]:
# Normalize the data
x_train_trans, x_test_trans = x_train_trans / 255, x_test_trans / 255

### Flatten the images and try ordinary DNN first

In [ ]:
# Build the model
dnn_scratch = Classifier(softmax_cross_entropy, softmax_cross_entropy_prime)
#dnn_scratch.add_layer(ConvolutionalLayer(16, 3))
#dnn_scratch.add_layer(MaxPoolingLayer(2))
dnn_scratch.add_layer(FullyConnectedLayer(196, 16))
dnn_scratch.add_layer(ActivationLayer(ReLU, ReLU_prime))
dnn_scratch.add_layer(FullyConnectedLayer(16, 5))

In [ ]:
X_train = x_train_trans.reshape(len(x_train_trans), -1)

# Fit the model
error = dnn_scratch.train(X_train, y_train, 10, 5e-4)

In [ ]:
X_test = x_test_trans.reshape(len(x_test_trans), -1)
y_test_pred = dnn_scratch.predict(X_test)
acc_rate = accuracy(y_test, y_test_pred)
print('Accuracy:', acc_rate)

### Try to build a CNN model

In [17]:
# Build the CNN model
cnn_scratch = Classifier(softmax_cross_entropy, softmax_cross_entropy_prime)
cnn_scratch.add_layer(ConvolutionalLayer(16, 3))
cnn_scratch.add_layer(MaxPoolingLayer(2))
cnn_scratch.add_layer(FlattenLayer())
cnn_scratch.add_layer(FullyConnectedLayer(576, 16))
cnn_scratch.add_layer(ActivationLayer(ReLU, ReLU_prime))
cnn_scratch.add_layer(FullyConnectedLayer(16, 5))

In [18]:
# Fit the model
error = cnn_scratch.train(x_train_trans, y_train, 10, 5e-4)

Streaming output truncated to the last 5000 lines.
true: 2, pred: 2
10-7501th sample
true: 0, pred: 0
10-7502th sample
true: 0, pred: 0
10-7503th sample
true: 0, pred: 0
10-7504th sample
true: 2, pred: 2
10-7505th sample
true: 4, pred: 4
10-7506th sample
true: 4, pred: 4
10-7507th sample
true: 1, pred: 1
10-7508th sample
true: 2, pred: 0
10-7509th sample
true: 3, pred: 3
10-7510th sample
true: 4, pred: 4
10-7511th sample
true: 4, pred: 4
10-7512th sample
true: 1, pred: 1
10-7513th sample
true: 4, pred: 4
10-7514th sample
true: 0, pred: 0
10-7515th sample
true: 1, pred: 1
10-7516th sample
true: 1, pred: 1
10-7517th sample
true: 3, pred: 3
10-7518th sample
true: 0, pred: 0
10-7519th sample
true: 3, pred: 2
10-7520th sample
true: 3, pred: 3
10-7521th sample
true: 2, pred: 4
10-7522th sample
true: 0, pred: 2
10-7523th sample
true: 3, pred: 3
10-7524th sample
true: 3, pred: 3
10-7525th sample
true: 2, pred: 2
10-7526th sample
true: 2, pred: 2
10-7527th sample
true: 3, pred: 1
10-7528th samp

In [19]:
y_test_pred_c = cnn_scratch.predict(x_test_trans)
acc_rate_c = accuracy(y_test, y_test_pred_c)
print('Accuracy:', acc_rate_c)

Accuracy: 0.8468


## We also try PyTorch and compare the result




DNN

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(196, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 5)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Prepare the data
X_train = torch.tensor(x_train_trans, dtype=torch.float32)
Y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(x_test_trans, dtype=torch.float32)
Y_test = torch.tensor(y_test, dtype=torch.float32)

# Create data loaders
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create the model
model = Net()

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    print('Epoch', epoch + 1)
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Flatten the data
        data = data.view(data.size(0), -1)

        # Forward pass
        optimizer.zero_grad()
        output = model(data)

        # Calculate the loss
        loss = loss_fn(output, torch.argmax(target, dim=1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Batch', batch_idx, 'Loss:', loss.item())

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        x_test_flat = X_test.view(X_test.size(0), -1)
        logits = model(x_test_flat)
        predicted_labels = torch.argmax(logits, dim=1)
        acc = (predicted_labels == torch.argmax(Y_test, dim=1)).float().mean()
        print('Validation Accuracy:', acc.item())

Epoch 1
Batch 0 Loss: 1.6093637943267822
Batch 100 Loss: 1.148697018623352
Batch 200 Loss: 0.6922160387039185
Batch 300 Loss: 0.39900875091552734
Validation Accuracy: 0.9323999881744385
Epoch 2
Batch 0 Loss: 0.5859517455101013
Batch 100 Loss: 0.2738678455352783
Batch 200 Loss: 0.3081545829772949
Batch 300 Loss: 0.2647506892681122
Validation Accuracy: 0.9476000070571899
Epoch 3
Batch 0 Loss: 0.24007831513881683
Batch 100 Loss: 0.20624297857284546
Batch 200 Loss: 0.3428870439529419
Batch 300 Loss: 0.16096876561641693
Validation Accuracy: 0.9571999907493591
Epoch 4
Batch 0 Loss: 0.12155663222074509
Batch 100 Loss: 0.20314303040504456
Batch 200 Loss: 0.18839578330516815
Batch 300 Loss: 0.1424766331911087
Validation Accuracy: 0.9635999798774719
Epoch 5
Batch 0 Loss: 0.09538808465003967
Batch 100 Loss: 0.2036803960800171
Batch 200 Loss: 0.25224190950393677
Batch 300 Loss: 0.24175788462162018
Validation Accuracy: 0.9639999866485596
Epoch 6
Batch 0 Loss: 0.07113588601350784
Batch 100 Loss: 0.0

In [21]:
### CNN